In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy
from ALLSampler_Sentence import SamplingSimulator, ModelSamplingSimulator

In [2]:
logger.remove()
logger.add(sys.stderr, level='DEBUG')

1

## Initiate Brat Reader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read Brat annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


## Define sampling function

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

## CRF Wrapper

In [10]:
from CRFWrapper_Sentence import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel


In [11]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Strength', 'Route', 'Reason', 'Drug', 'ADE', 'Dosage', 'Form', 'Frequency', 'Duration'}


In [12]:
crf_model=CRFModel(anno_types=annos)

## Convert training and testing docs into sentence level dataframe

In [13]:
_, train_df=convert_docs(train_docs, anno_types=annos)

In [14]:
_, test_df=convert_docs(test_docs, anno_types=annos)

In [ ]:
train_df

In [16]:
train_df.label.unique()

array(['O', 'B-Drug', 'B-Route', 'B-Reason', 'I-Reason', 'B-Strength',
       'I-Strength', 'B-Frequency', 'B-Duration', 'I-Duration', 'B-Form',
       'B-Dosage', 'I-Dosage', 'B-ADE', 'I-Frequency', 'I-Drug', 'I-Form',
       'I-Route', 'I-ADE'], dtype=object)

## Bootstrap 3 different initials runs

In [17]:
boostrap_runs=3
total_round=10
logger.remove()
logger.add(sys.stderr, level='INFO')

2

In [18]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [19]:
%%time
all_scores=[]
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    crf_model=CRFModel(anno_types=annos, topNUncertainToken=400)
    m_simulator=ModelSamplingSimulator(total_sents=train_df, 
                                   total_round=10, 
                                   modelWrapper=crf_model, 
                                   eval_sents=test_df, 
                                   init_seed=seed,
                                   sample_all_on_last_round=False)  
    m_simulator.num_per_round=200
    scores=m_simulator.simulate_rounds(boostrap_times=500)
    all_scores.append(scores) 

2024-07-15 22:34:53.503 | INFO     | __main__:<module>:3 - start run 0.
2024-07-15 22:34:53.607 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 0.
2024-07-15 22:34:53.995 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 200, remaining sentences: 51598
2024-07-15 22:44:25.463 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 1.
2024-07-15 22:45:23.629 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 400, remaining sentences: 51398
2024-07-15 22:54:27.000 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 2.
2024-07-15 22:55:24.643 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 600, remaining sentences: 51198
2024-07-15 23:04:22.157 | INFO     | ALLSampler_Sentence:simulate_rounds:99 - simulate round 3.
2024-07-15 23:05:21.043 | INFO     | ALLSampler_Sentence:keep_sample:89 - current sampled sentences: 800, remaining sentences: 50998
2024-07-15 2

CPU times: total: 4h 58min 36s
Wall time: 4h 58min 42s


In [20]:
joblib.dump(all_scores, r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_full500bootstrap.joblib')


['../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling_full500bootstrap.joblib']

 ## Debug